In [ ]:
# Importações para treinamento do modelo
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, concatenate, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import joblib
import warnings
warnings.filterwarnings('ignore')

# Configurar GPU se disponível
print("🔧 Configurando ambiente TensorFlow...")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU disponível: {len(gpus)} dispositivo(s)")
    except RuntimeError as e:
        print(f"⚠️ Erro configurando GPU: {e}")
else:
    print("💻 Executando em CPU")

# Configurar seed para reprodutibilidade
tf.random.set_seed(42)
np.random.seed(42)

print("✅ Ambiente configurado!")


In [ ]:
# Carregar dados preprocessados com features atmosféricas
print("📊 Carregando dados para treinamento...")

# Componente 1: Historical Forecast (149 variáveis)
try:
    df_forecast = pd.read_csv('../data/processed/atmospheric_features_149vars.csv')
    df_forecast['datetime'] = pd.to_datetime(df_forecast['datetime'])
    df_forecast.set_index('datetime', inplace=True)
    print(f"✅ Componente 1 (Forecast): {df_forecast.shape}")
except FileNotFoundError:
    print("❌ Features atmosféricas não encontradas. Tentando dados processados...")
    try:
        # Tentar carregar dados processados gerais
        from pathlib import Path
        processed_files = list(Path("../data/processed/").glob("*.csv"))
        if processed_files:
            print(f"📁 Encontrados {len(processed_files)} arquivos processados")
            # Pegar o maior arquivo (provavelmente o principal)
            largest_file = max(processed_files, key=lambda f: f.stat().st_size)
            df_forecast = pd.read_csv(largest_file)
            if 'datetime' in df_forecast.columns:
                df_forecast['datetime'] = pd.to_datetime(df_forecast['datetime'])
                df_forecast.set_index('datetime', inplace=True)
            print(f"✅ Dados carregados de {largest_file.name}: {df_forecast.shape}")
        else:
            df_forecast = None
            print("❌ Nenhum dado processado encontrado")
    except Exception as e:
        print(f"❌ Erro: {e}")
        df_forecast = None

# Componente 2: Historical Weather (25 variáveis)
try:
    df_weather = pd.read_csv('../data/processed/surface_features_25vars.csv')
    df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
    df_weather.set_index('datetime', inplace=True)
    print(f"✅ Componente 2 (Weather): {df_weather.shape}")
except FileNotFoundError:
    print("⚠️ Componente 2 não disponível - usando apenas Componente 1")
    df_weather = None

# Configurações do modelo híbrido (adaptativas baseadas nos dados disponíveis)
if df_forecast is not None:
    n_features_1 = df_forecast.shape[1]
    print(f"📊 Features disponíveis no Componente 1: {n_features_1}")
else:
    n_features_1 = 149  # valor padrão

if df_weather is not None:
    n_features_2 = df_weather.shape[1]
    print(f"📊 Features disponíveis no Componente 2: {n_features_2}")
else:
    n_features_2 = 25  # valor padrão

HYBRID_CONFIG = {
    'component_1': {
        'sequence_length': 48,  # 48 horas
        'features': n_features_1,
        'weight': 0.7,
        'lstm_units': [256, 128, 64],
        'dropout': 0.2
    },
    'component_2': {
        'sequence_length': 72,  # 72 horas  
        'features': n_features_2,
        'weight': 0.3,
        'lstm_units': [128, 64, 32],
        'dropout': 0.3
    },
    'target_variable': 'precipitation',  # Variável alvo
    'forecast_horizon': 24  # Previsão de 24h à frente
}

print(f"🎯 Configuração híbrida: {HYBRID_CONFIG['component_1']['weight']:.1f}/{HYBRID_CONFIG['component_2']['weight']:.1f}")
print(f"📈 Horizonte de previsão: {HYBRID_CONFIG['forecast_horizon']}h")
print(f"🔧 Features C1: {HYBRID_CONFIG['component_1']['features']}, C2: {HYBRID_CONFIG['component_2']['features']}")


In [ ]:
def create_hybrid_lstm_model(config):
    """
    Cria modelo LSTM híbrido com dois componentes:
    - Componente 1: Dados atmosféricos completos (149 vars)
    - Componente 2: Dados de superfície (25 vars)
    """
    
    # Componente 1: Atmospheric Features (149 variables)
    input_1 = Input(shape=(config['component_1']['sequence_length'], 
                          config['component_1']['features']), 
                   name='atmospheric_input')
    
    # LSTM stack para componente atmosférico
    lstm_1 = LSTM(config['component_1']['lstm_units'][0], 
                  return_sequences=True, 
                  name='lstm_1_layer_1')(input_1)
    lstm_1 = Dropout(config['component_1']['dropout'])(lstm_1)
    
    lstm_1 = LSTM(config['component_1']['lstm_units'][1], 
                  return_sequences=True,
                  name='lstm_1_layer_2')(lstm_1)
    lstm_1 = Dropout(config['component_1']['dropout'])(lstm_1)
    
    lstm_1 = LSTM(config['component_1']['lstm_units'][2], 
                  return_sequences=False,
                  name='lstm_1_layer_3')(lstm_1)
    lstm_1 = Dropout(config['component_1']['dropout'])(lstm_1)
    
    # Dense layer para componente 1
    dense_1 = Dense(32, activation='relu', name='dense_1')(lstm_1)
    output_1 = Dense(1, activation='linear', name='output_1')(dense_1)
    
    # Componente 2: Surface Features (25 variables)
    input_2 = Input(shape=(config['component_2']['sequence_length'], 
                          config['component_2']['features']), 
                   name='surface_input')
    
    # LSTM stack para componente de superfície
    lstm_2 = LSTM(config['component_2']['lstm_units'][0], 
                  return_sequences=True,
                  name='lstm_2_layer_1')(input_2)
    lstm_2 = Dropout(config['component_2']['dropout'])(lstm_2)
    
    lstm_2 = LSTM(config['component_2']['lstm_units'][1], 
                  return_sequences=True,
                  name='lstm_2_layer_2')(lstm_2)
    lstm_2 = Dropout(config['component_2']['dropout'])(lstm_2)
    
    lstm_2 = LSTM(config['component_2']['lstm_units'][2], 
                  return_sequences=False,
                  name='lstm_2_layer_3')(lstm_2)
    lstm_2 = Dropout(config['component_2']['dropout'])(lstm_2)
    
    # Dense layer para componente 2
    dense_2 = Dense(16, activation='relu', name='dense_2')(lstm_2)
    output_2 = Dense(1, activation='linear', name='output_2')(dense_2)
    
    # Ensemble: Weighted Average
    def weighted_average(inputs):
        output_1, output_2 = inputs
        weight_1 = config['component_1']['weight']
        weight_2 = config['component_2']['weight']
        return weight_1 * output_1 + weight_2 * output_2
    
    # Combinar saídas
    from tensorflow.keras.layers import Lambda
    ensemble_output = Lambda(weighted_average, name='ensemble_output')([output_1, output_2])
    
    # Criar modelo
    model = Model(inputs=[input_1, input_2], outputs=ensemble_output, name='hybrid_lstm_model')
    
    return model

# Criar modelo híbrido
print("🏗️ Construindo arquitetura LSTM híbrida...")
hybrid_model = create_hybrid_lstm_model(HYBRID_CONFIG)

# Compilar modelo
hybrid_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae', 'mse']
)

# Visualizar arquitetura
print("📋 Arquitetura do modelo:")
hybrid_model.summary()

# Plotar arquitetura
try:
    tf.keras.utils.plot_model(
        hybrid_model, 
        to_file='models/hybrid_lstm_architecture.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB',
        expand_nested=True
    )
    print("✅ Arquitetura salva em: models/hybrid_lstm_architecture.png")
except Exception as e:
    print(f"⚠️ Não foi possível plotar arquitetura: {e}")
